# 1. Okay we've explored lots of functionality which really brings the MITRE data to life and combines it in targetted, useful ways.

# In this last exercise we're going to tie it all togehter into one big script.

# We're also going to add lot's of bells and whistles to improve:

# * Search
# * Report formatting
# * Different levels of report detail
# * A dedicated "Software" report
# * Inclusion of MITRE tactic descriptions

# So let's begin with installing the dependencies - Theres nothing new here.

In [ ]:
!pip install markdown2 pdfkit rapidfuzz
!apt update
!apt install wkhtmltopdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.9 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages

# 2. Next we'll import our libraries, again, there's nothing new here.

In [ ]:
import os
import json
import requests
import pandas as pd
import markdown2
import pdfkit
from rapidfuzz import process

# 3. Here we'll prepare our data as normal but we'll also include a lookup table to enrich MITRE tactics with descriptions, which are abscent from the data we're working from.

In [ ]:
# --- Setup: Load and Prepare STIX Data ---
url = "https://raw.githubusercontent.com/mitre/cti/master/enterprise-attack/enterprise-attack.json"
stix_data = requests.get(url).json()
all_objects_df = pd.DataFrame(stix_data["objects"])

relationships_df = all_objects_df[all_objects_df["type"] == "relationship"].copy()
threat_actors_df = all_objects_df[all_objects_df["type"] == "intrusion-set"].copy()
campaigns_df = all_objects_df[all_objects_df["type"] == "campaign"].copy()
techniques_df = all_objects_df[all_objects_df["type"] == "attack-pattern"].copy()
software_df = all_objects_df[all_objects_df["type"].isin(["tool", "malware"])].copy()
mitigations_df = all_objects_df[all_objects_df["type"] == "course-of-action"].copy()

# --- Tactic Descriptions ---
tactic_descriptions = {
    "reconnaissance": "Active and passive information gathering about a target.",
    "resource-development": "Establishing resources to support operations.",
    "initial-access": "Gaining entry into a network or system.",
    "execution": "Running malicious code.",
    "persistence": "Maintaining access through restarts or credential changes.",
    "privilege-escalation": "Gaining higher-level permissions.",
    "defense-evasion": "Avoiding detection and prevention.",
    "credential-access": "Stealing account credentials.",
    "discovery": "Identifying system or network information.",
    "lateral-movement": "Moving through the network after initial compromise.",
    "collection": "Gathering data of interest.",
    "command-and-control": "Communicating with compromised systems.",
    "exfiltration": "Stealing and transporting data out of the network.",
    "impact": "Manipulating, interrupting, or destroying systems or data."
}

# 4. Next up is our helper functions and there are alot of new ones here. The important new additions are:
# * `sanitize_for_markdown`
### Which will sanitize characters which could interfere with our report generation.
# * `list_all_actors/software/campaigns`
### These functions act as search helpers. In our previous searches the tool generated a report based on the first match. These allow you to choose from a list of all actors, campaigns and software in the database - if a search fails to find what you're looking for.
# * `search_entities`
### Although we had a search function in the last lab this one is improved and allows you to limit search results from one specific catagory.

In [ ]:
# --- Utility Functions ---
def get_description(obj):
    desc = obj.get("description", "No description available.")
    return "\n".join(desc) if isinstance(desc, list) else desc

def sanitize_for_markdown(text):
    if not isinstance(text, str):
        return text
    import re
    text = re.sub(r"<code>(.*?)</code>", r"`\1`", text, flags=re.DOTALL)
    special_chars = {'\\': r'\\', '&': r'&amp;', '%': r'\%', '$': r'\$', '#': r'\#',
                     '_': r'\_', '{': r'\{', '}': r'\}', '~': r'\textasciitilde{}', '^': r'\textasciicircum{}'}
    for char, escaped in special_chars.items():
        text = text.replace(char, escaped)
    return text.replace("javascript:", "javascript&#58;")

def get_external_references(stix_obj):
    return [ref.get("url") for ref in stix_obj.get("external_references", []) if ref.get("url")]

def get_tactics_from_technique(tech):
    return [phase["phase_name"] for phase in tech.get("kill_chain_phases", []) if isinstance(phase, dict)]

# --- Relationship Queries ---
def get_techniques_related_to(object_id):
    rels = relationships_df[(relationships_df["source_ref"] == object_id) & relationships_df["target_ref"].str.startswith("attack-pattern")]
    return techniques_df[techniques_df["id"].isin(rels["target_ref"])]

def get_campaigns_by_actor_id(actor_id):
    rels = relationships_df[(relationships_df["target_ref"] == actor_id) & relationships_df["source_ref"].str.contains("campaign")]
    return campaigns_df[campaigns_df["id"].isin(rels["source_ref"])]

def get_software_by_campaign_id(campaign_id):
    rels = relationships_df[(relationships_df["source_ref"] == campaign_id) & relationships_df["target_ref"].str.startswith(("tool", "malware"))]
    return software_df[software_df["id"].isin(rels["target_ref"])]

def get_software_used_by_actor_id(actor_id):
    rels = relationships_df[
        ((relationships_df["source_ref"] == actor_id) & relationships_df["target_ref"].str.startswith(("tool", "malware"))) |
        ((relationships_df["target_ref"] == actor_id) & relationships_df["source_ref"].str.startswith(("tool", "malware")))
    ]
    ids = set(rels["source_ref"]).union(rels["target_ref"])
    return software_df[software_df["id"].isin(ids)][["id", "name"]]

def get_techniques_by_campaign_id(campaign_id):
    rels = relationships_df[(relationships_df["source_ref"] == campaign_id) & relationships_df["target_ref"].str.startswith("attack-pattern")]
    return techniques_df[techniques_df["id"].isin(rels["target_ref"])]

def get_techniques_by_software_id(software_id):
    rels = relationships_df[(relationships_df["source_ref"] == software_id) & relationships_df["target_ref"].str.startswith("attack-pattern")]
    return techniques_df[techniques_df["id"].isin(rels["target_ref"])]

def get_mitigations_by_technique_id(technique_id):
    rels = relationships_df[(relationships_df['target_ref'] == technique_id) & (relationships_df['source_ref'].str.contains('course-of-action'))]
    return mitigations_df[mitigations_df['id'].isin(rels['source_ref'])]

# --- Lookup Helpers ---
def find_actor_id_by_external_id_or_name(identifier):
    match = threat_actors_df[
        threat_actors_df["external_references"].apply(lambda refs: any(ref.get("external_id") == identifier for ref in refs if isinstance(ref, dict))) |
        (threat_actors_df["name"].str.lower() == identifier.lower())
    ]
    return match.iloc[0]["id"] if not match.empty else None

# --- Search & Listings ---
def list_all_software():
    return sorted(software_df["name"].dropna().unique())

def list_all_actors():
    return sorted(threat_actors_df["name"].dropna().unique())

def list_all_campaigns():
    return sorted(campaigns_df["name"].dropna().unique())

def get_software_by_tactic(tactic_filter):
    matches = set()
    for _, sw in software_df.iterrows():
        for _, tech in get_techniques_by_software_id(sw["id"]).iterrows():
            if tactic_filter.lower() in (t.lower() for t in get_tactics_from_technique(tech)):
                matches.add(sw["name"])
    return sorted(matches)

def search_entities(query, search_type="all", limit=10):
    results = {}
    if search_type in ["software", "all"]:
        results["software"] = [match[0] for match in process.extract(query, list_all_software(), limit=limit, score_cutoff=60)]
    if search_type in ["actor", "all"]:
        results["actors"] = [match[0] for match in process.extract(query, list_all_actors(), limit=limit, score_cutoff=60)]
    if search_type in ["campaign", "all"]:
        results["campaigns"] = [match[0] for match in process.extract(query, list_all_campaigns(), limit=limit, score_cutoff=60)]
    return results

# 5. Next we have our `generate_threat_actor_report` which has increased massively in size to include formatting improvements, variable levels of detail and also the ability to represent techniques and tactics in a table along with their tactic descriptions from the lookup table we defined earlier.

#

In [ ]:
# --- Report Generation ---

def generate_threat_actor_report(actor_external_id, verbosity="executive", output_path="report.md"):
    actor_row = threat_actors_df[
        threat_actors_df["external_references"].apply(lambda refs: any(
            ref.get("external_id") == actor_external_id for ref in refs if isinstance(ref, dict)))
    ]

    if actor_row.empty:
        print(f"Threat actor with ID {actor_external_id} not found.")
        return

    actor = actor_row.iloc[0]
    actor_name = actor["name"]
    actor_id = actor["id"]

    lines = [f"**Threat Actor Report:** {actor_name} ({actor_external_id}) <br>"]
    lines.append(f"**Description:** {sanitize_for_markdown(get_description(actor))} <br>")

    techniques = get_techniques_related_to(actor_id)

    if verbosity == "executive":
        tactic_set = set()
        for _, tech in techniques.iterrows():
            tactic_set.update(get_tactics_from_technique(tech))
        lines.append("<br>**Tactics Observed**<br>")
        for tactic in sorted(tactic_set):
            lines.append(f"{tactic} - ")
    else:
        lines.append("<div class='page-break'></div>")

## Techniques Used
        lines.append("<table><thead><tr><th>Technique ID</th><th>Name</th><th>Tactics</th></tr></thead><tbody>")
        for _, tech in techniques.iterrows():
            tactic_names = get_tactics_from_technique(tech)
            tid = next((ref.get("external_id") for ref in tech.get("external_references", []) if ref.get("source_name") == "mitre-attack"), None)
            if tid:
                url = f"https://attack.mitre.org/techniques/{tid.replace('.', '/')}"
                lines.append(f"<tr><td><a href='{url}'>{tid}</a></td><td>{tech['name']}</td><td>{', '.join(tactic_names)}</td></tr>")
        lines.append("</tbody></table>")

    lines.append("<div class='page-break'></div>")

## Campaigns
    campaigns = get_campaigns_by_actor_id(actor_id)

    for _, campaign in campaigns.iterrows():
        lines.append(f"<div class='page-break'></div><br>**{campaign['name']}**<br>")
        lines.append("<br>")
        lines.append(f"**Description:** {sanitize_for_markdown(get_description(campaign))}<br>")

        refs = get_external_references(campaign)
        if refs:
            lines.append("**References:**<br>")
            for ref in refs:
                lines.append(f"- [{ref}]({ref})<br>")

        if verbosity == "analyst":
            campaign_techniques = get_techniques_by_campaign_id(campaign["id"])
            if not campaign_techniques.empty:
                lines.append("<br>**Techniques Used in Campaign:**")
                lines.append("<table><thead><tr><th>Technique ID</th><th>Name</th><th>Tactics</th></tr></thead><tbody>")
                for _, tech in campaign_techniques.iterrows():
                    tactic_names = get_tactics_from_technique(tech)
                    tid = next((ref.get("external_id") for ref in tech.get("external_references", []) if ref.get("source_name") == "mitre-attack"), None)
                    if tid:
                        url = f"https://attack.mitre.org/techniques/{tid.replace('.', '/')}"
                        lines.append(f"<tr><td><a href='{url}'>{tid}</a></td><td>{tech['name']}</td><td>{', '.join(tactic_names)}</td></tr>")
                lines.append("</tbody></table>")

            campaign_software = get_software_by_campaign_id(campaign["id"])
            for _, sw in campaign_software.iterrows():
                lines.append(f"<div class='page-break'></div>**Tool: {sw['name']}**")
                lines.append(f"<br> **Description:** {sanitize_for_markdown(sw.get('description', 'No description'))}")
                sw_techniques = get_techniques_by_software_id(sw["id"])
                if not sw_techniques.empty:
                    lines.append("<table><thead><tr><th>Technique ID</th><th>Name</th><th>Tactics</th></tr></thead><tbody>")
                    for _, tech in sw_techniques.iterrows():
                        tactic_names = get_tactics_from_technique(tech)
                        tid = next((ref.get("external_id") for ref in tech.get("external_references", []) if ref.get("source_name") == "mitre-attack"), None)
                        if tid:
                            url = f"https://attack.mitre.org/techniques/{tid.replace('.', '/')}"
                            lines.append(f"<tr><td><a href='{url}'>{tid}</a></td><td>{tech['name']}</td><td>{', '.join(tactic_names)}</td></tr>")
                    lines.append("</tbody></table>")

    if verbosity == "analyst":
        actor_software = get_software_used_by_actor_id(actor_id)
        lines.append("<div class='page-break'></div>")

## Tools and Malware Used
        if actor_software.empty:
            lines.append("No tools or malware directly linked to this threat actor.")
        else:
            lines.append(f"**Tools used by {actor_name}**<br>")
            for _, sw in actor_software.iterrows():
                lines.append(f"**{sw['name']}**<br>")
                sw_techniques = get_techniques_by_software_id(sw['id'])
                if not sw_techniques.empty:
                    lines.append("<table><thead><tr><th>Technique ID</th><th>Name</th><th>Tactics</th></tr></thead><tbody>")
                    for _, tech in sw_techniques.iterrows():
                        tactic_names = get_tactics_from_technique(tech)
                        tid = next((ref.get("external_id") for ref in tech.get("external_references", []) if ref.get("source_name") == "mitre-attack"), None)
                        if tid:
                            url = f"https://attack.mitre.org/techniques/{tid.replace('.', '/')}"
                            lines.append(f"<tr><td><a href='{url}'>{tid}</a></td><td>{tech['name']}</td><td>{', '.join(tactic_names)}</td></tr>")
                    lines.append("</tbody></table>")

    html = markdown2.markdown("".join(lines), extras=["fenced-code-blocks", "tables"])
    pdfkit.from_string(html, output_path.replace(".md", ".pdf"), options={
        'enable-local-file-access': None,
        'user-style-sheet': "/content/report.css"
    })

    with open(output_path, "w", encoding="utf-8") as f:
        f.write("".join(lines))

# 6. Next you'll see an entirely new report format the `generate_software_report`.

# The idea behind this was to cut down on the clutter in the threat actor report.

# Imagine you're looking at an actor and you become interested in one particular tool that they use - you can run this report to generate a detailed look into that tool along with all the mitigations needed to protect against that tool.

In [ ]:

def generate_software_report(software_external_id, output_path="software_report.md"):
    sw_row = software_df[software_df["external_references"].apply(lambda refs: any(ref.get("external_id") == software_external_id for ref in refs if isinstance(ref, dict)))]
    if sw_row.empty:
        print(f"Software with ID {software_external_id} not found.")
        return

    sw = sw_row.iloc[0]
    lines = [f"# Software Report: {sw['name']} ({software_external_id})\n"]
    techniques = get_techniques_by_software_id(sw["id"])
    tactic_set = set()
    phase_set = set()
    lockheed_map = {
        "reconnaissance": ["reconnaissance"],
        "resource-development": ["weaponization"],
        "initial-access": ["delivery"],
        "execution": ["exploitation"],
        "privilege-escalation": ["exploitation"],
        "defense-evasion": ["exploitation"],
        "persistence": ["installation"],
        "command-and-control": ["command and control"],
        "collection": ["actions on objectives"],
        "exfiltration": ["actions on objectives"],
        "impact": ["actions on objectives"]
    }
    for _, tech in techniques.iterrows():
        phases = tech.get("kill_chain_phases", [])
        for phase in phases:
            if isinstance(phase, dict):
                tactic = phase.get("phase_name")
                tactic_set.add(tactic)
                if tactic in lockheed_map:
                    phase_set.update(lockheed_map[tactic])

    if tactic_set:
        lines.append(f"**Tactics:** {', '.join(sorted(tactic_set))}")
    if phase_set:
        lines.append(f"<br>**Lockheed Kill Chain Phases:** {', '.join(sorted(phase_set))}")
    lines.append(f"<br>**Description:** {sanitize_for_markdown(get_description(sw))}")
    lines.append("")  # adds a blank line

    lines.append("<h2>Used By Threat Actors</h2>")
    used_by = relationships_df[
        (relationships_df["target_ref"] == sw["id"]) &
        (relationships_df["source_ref"].str.contains("intrusion-set"))
    ]
    actor_ids = used_by["source_ref"].tolist()
    actor_names = threat_actors_df[threat_actors_df["id"].isin(actor_ids)]["name"].tolist()
    for name in sorted(actor_names):
        lines.append(f"- {name}")

    lines.append("<div class='page-break'></div><h2>Techniques Used by Software</h2>")
    for _, tech in techniques.iterrows():
        tid = next((ref.get("external_id") for ref in tech.get("external_references", []) if ref.get("source_name") == "mitre-attack"), None)
        if not tid:
            continue
        tactic_names = get_tactics_from_technique(tech)
        url = f"https://attack.mitre.org/techniques/{tid.replace('.', '/')}"
        lines.append(f"<h3>Technique: <a href='{url}'>{tech['name']}</a> ({tid})</h3>")
        lines.append(f"**Tactics:** {', '.join(tactic_names)}<br>")
        lines.append(f"**Description:** {sanitize_for_markdown(get_description(tech))}")
        lines.append("")  # adds a blank line


        mitigations = get_mitigations_by_technique_id(tech["id"])
        if not mitigations.empty:
            lines.append("<br><br>**Mitigations:**<br>")
            for _, mit in mitigations.iterrows():
                mit_name = mit.get("name", "Unnamed")
                mit_desc = sanitize_for_markdown(get_description(mit))
                lines.append(f"**{mit_name}**: {mit_desc}<br><br>")
        else:
            lines.append("<p><strong>No mitigations found.</strong></p>")

    with open(output_path, "w") as f:
        f.write("".join(lines))

    html = markdown2.markdown("".join(lines), extras=["fenced-code-blocks", "tables"])
    pdfkit.from_string(html, output_path.replace(".md", ".pdf"), options={
        'enable-local-file-access': None,
        'user-style-sheet': "/content/report.css"
    })


# 7. This cell includes a helper function to translate search results into entity IDs. Our previous functions accept entity IDs in order to start generating a report. Either an actor ID, campaign ID or software ID.

# The really key function here is the `run_interactive_report_generator`.

# This will interact with the user to perform a search on a specific entity catagory, list out all entities in that category if you can't find your actor in the search results and generate a report based on the level of detail you require.

# One thing to note is that there isn't a separate report for campaign searches. The campaign search will find the actor associated with that campaign and then generate a threat actor report, as though you'd searched for an actor.

In [ ]:
# --- Interactive Search and Report Generation ---

def get_external_id_by_name(name, entity_type="actor"):
    df_lookup = {
        "actor": threat_actors_df,
        "software": software_df,
        "campaign": campaigns_df
    }
    df = df_lookup.get(entity_type)
    if df is None:
        print(f"❌ Unknown entity type: {entity_type}")
        return None
    row = df[df["name"].str.lower() == name.lower()]
    if row.empty:
        print(f"❌ No match found for name: {name}")
        return None
    refs = row.iloc[0].get("external_references", [])
    return next((ref.get("external_id") for ref in refs if ref.get("external_id")), None)

def get_actor_id_by_campaign_name(campaign_name):
    campaign_row = campaigns_df[campaigns_df["name"].str.lower() == campaign_name.lower()]
    if campaign_row.empty:
        print(f"❌ No campaign found with name: {campaign_name}")
        return None
    campaign_id = campaign_row.iloc[0]["id"]
    rels = relationships_df[
        (relationships_df["source_ref"] == campaign_id) &
        (relationships_df["target_ref"].str.contains("intrusion-set"))
    ]
    if rels.empty:
        print("❌ No linked actor found for this campaign.")
        return None
    actor_id = rels.iloc[0]["target_ref"]
    actor_row = threat_actors_df[threat_actors_df["id"] == actor_id]
    if actor_row.empty:
        return None
    refs = actor_row.iloc[0].get("external_references", [])
    return next((ref.get("external_id") for ref in refs if ref.get("external_id")), None)

def run_interactive_report_generator():
    import re
    print("\nWhat type of entity would you like to search?")
    print("1. Actor")
    print("2. Campaign")
    print("3. Software")
    choice = input("Enter 1, 2, or 3: ").strip()

    type_map = {"1": "actor", "2": "campaign", "3": "software"}
    entity_type = type_map.get(choice)
    if not entity_type:
        print("Invalid selection.")
        return

    query = input(f"Enter search term for {entity_type}: ").strip()
    results = search_entities(query.lower(), search_type=entity_type)
    key = "software" if entity_type == "software" else f"{entity_type}s"
    items = results.get(key, [])
    if not items:
        print("No results found.")
        return

    print("Search Results:")
    for i, item in enumerate(items, start=1):
        print(f"{i}. {item}")
    print(f"{len(items)+1}. List all in category")

    sel = input("Enter the number of the item to generate a report: ").strip()
    if sel == str(len(items)+1):
        print("Listing all available items in this category:")
        full_list_func = {
            "actor": list_all_actors,
            "campaign": list_all_campaigns,
            "software": list_all_software
        }.get(entity_type)
        full_list = full_list_func()
        for i, name in enumerate(full_list, start=1):
            print(f"{i}. {name}")

        sel2 = input("Enter the number of the item to generate a report: ").strip()
        if not sel2.isdigit() or not (1 <= int(sel2) <= len(full_list)):
            print("Invalid selection.")
            return
        selected_name = full_list[int(sel2)-1]
    else:
        if not sel.isdigit() or not (1 <= int(sel) <= len(items)):
            print("Invalid selection.")
            return
        selected_name = items[int(sel)-1]

    # === Unified report generation logic ===
    if entity_type in ["actor", "campaign"]:
        if entity_type == "campaign":
            external_id = get_actor_id_by_campaign_name(selected_name)
            if not external_id:
                print("Unable to resolve actor from campaign.")
                return
        else:
            external_id = get_external_id_by_name(selected_name, entity_type="actor")

        verbosity = input("Level of detail (executive/analyst): ").strip().lower()
        if verbosity not in ["executive", "analyst"]:
            verbosity = "executive"
        output_file = f"report_{external_id}_{verbosity}.md"
        generate_threat_actor_report(external_id, verbosity=verbosity, output_path=output_file)

    elif entity_type == "software":
        external_id = get_external_id_by_name(selected_name, entity_type="software")
        generate_software_report(external_id)

    print("\n✅ Report generated successfully!")
    return

    selected_name = items[int(sel)-1]

    if entity_type == "campaign":
        external_id = get_actor_id_by_campaign_name(selected_name)
        if not external_id:
            print("Unable to resolve actor from campaign.")
            return
        verbosity = input("Level of detail (executive/analyst): ").strip().lower()
        if verbosity not in ["executive", "analyst"]:
            verbosity = "executive"
        generate_threat_actor_report(external_id, verbosity=verbosity)
    elif entity_type == "actor":
        external_id = get_external_id_by_name(selected_name, entity_type="actor")
        verbosity = input("Level of detail (executive/analyst): ").strip().lower()
        if verbosity not in ["executive", "analyst"]:
            verbosity = "executive"
            generate_threat_actor_report(external_id, verbosity=verbosity)
    elif entity_type == "software":
        external_id = get_external_id_by_name(selected_name, entity_type="software")
        generate_software_report(external_id)

    print("\n✅ Report generated successfully!")


# 8. So this is where we run our code!

# This one function ties everything else we've covered together into one simple, easy to use interactive prompt.

# Use this cell to search for `campaign` and then `ukraine`.

# Use it to search for `software` and `cat`
# You'll notice that `mimicatz` isnt in the results - so choose the option to list all and try and find it.
# You should find it around line 348 - Can you see the mistake I made?
# Choose 348 and generate a report.

# Read through the reports. You'll notice I've included hyperlinks to extra resources. These links may point to 3rd party reporting on campaigns or point to MITRE to allow you to get right to the information you're looking for.

# I hope you enjoy using this tool but, more than that, I hope you enjoy making it your own.

# It has a few small bugs need ironing out and I hope you perfect the code and improve the formatting of the finished report!

In [ ]:
run_interactive_report_generator()


What type of entity would you like to search?
1. Actor
2. Campaign
3. Software
Enter 1, 2, or 3: 1
Enter search term for actor: sand worm
Search Results:
1. Sandworm Team
2. Orangeworm
3. List all in category
Enter the number of the item to generate a report: 1
Level of detail (executive/analyst): analyst

✅ Report generated successfully!
